In [7]:
from torch.utils.data import Dataset
import numpy as np
import pickle
import cv2
import torch
from fivesafe.object_detection import Detection_w_mask, draw_detection
from torch.utils.data import DataLoader
from torch.utils.data import random_split
from utils.utils import PadHull, ScaleToImage, FlattenCoordinates
from sklearn.pipeline import Pipeline, make_pipeline
from sklearn.preprocessing import StandardScaler
from dataset import VehiclePSIDataset
from tqdm.notebook import tqdm, trange

dataset = VehiclePSIDataset(base_url='/Users/tobias/ziegleto/data/5Safe/carla/etron')
train_size = int(0.8 * len(dataset))
test_size = len(dataset) - train_size
train_set, val_set = random_split(dataset, [train_size, test_size])
train_dataloader = DataLoader(train_set, batch_size=train_size, shuffle=True)
val_dataloader = DataLoader(val_set, batch_size=test_size, shuffle=True)
    
X_train, y_train = next(iter(train_dataloader))
X_val, y_val = next(iter(val_dataloader))
X_train = X_train.numpy()
y_train = y_train.numpy()
X_val = X_val.numpy()
y_val = y_val.numpy()
print(X_train.shape)
print(y_train.shape)
print(X_val.shape)
print(y_val.shape)

(498, 2, 65, 2)
(498, 2)
(125, 2, 65, 2)
(125, 2)


In [8]:
import pprint
pp = pprint.PrettyPrinter()
random_state = 42

In [10]:
import csv
from sklearn.linear_model import Ridge
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.multioutput import MultiOutputRegressor, RegressorChain
import joblib
#from utils.utils import fit_and_score, HardCodedEstimator

scaled_pipeline = make_pipeline(#PadHull(False, n_points_per_hull),
                               ScaleToImage(image_width, image_height),
                               FlattenCoordinates(n_points_per_hull),
                               StandardScaler()) # ridge needs standard scaler

non_scaled_pipeline = make_pipeline(#PadHull(False, n_points_per_hull),
                            ScaleToImage(image_width, image_height),
                            FlattenCoordinates(n_points_per_hull),
                            StandardScaler())

# commented out models don't work
classifiers = [
    (Ridge(random_state=random_state), scaled_pipeline),
    # RegressorChain(SVR()),
    # MultiOutputRegressor(SVR()),
    (RegressorChain(GradientBoostingRegressor(n_estimators=60, random_state=random_state)), non_scaled_pipeline),
    (MultiOutputRegressor(GradientBoostingRegressor(n_estimators=60, random_state=random_state)), non_scaled_pipeline),
    #RegressorChain(LinearSVR()),
    # MultiOutputRegressor(LinearSVR()),
    (DecisionTreeRegressor(max_depth=18, random_state=random_state), non_scaled_pipeline),
    #(HardCodedEstimator("homography_matrix.json", 1), None),
]
evaluation = fit_and_score(classifiers, X_train, y_train, random_state=random_state)
with open("./evaluation_baseline.csv", "w") as f:
    w = csv.DictWriter(f, evaluation[0].keys())
    w.writeheader()
    w.writerows(evaluation)
pp.pprint(evaluation)
#_=joblib.dump(evaluation, "classifiers.joblib")

  0%|          | 0/4 [00:00<?, ?it/s]

(996, 130)
(498, 2)


NameError: name 'train_test_split' is not defined

In [29]:
class FlattenCoordinates(BaseEstimator, TransformerMixin):
    def __init__(self, n_coordinates: int) -> None:
        super().__init__()
        self.n_coordinates = n_coordinates
        

    def fit(self, X):
        return self
    
    def transform(self, X):
        # flatten all coordinates so they can fit into the models
        return X.reshape(-1, self.n_coordinates*2)

NameError: name 'BaseEstimator' is not defined

In [28]:
#pipeline = make_pipeline(#PadHull(False, n_points_per_hull),
                          #  ScaleToImage(image_width, image_height),
                           ## FlattenCoordinates(n_points_per_hull),
                          #  StandardScaler())

#X_train_transformed = pipeline.fit_transform(X_train.copy())
#X_val_transformed = pipeline.fit_transform(X_val.copy())
print(X_train.shape)
sti = ScaleToImage(image_width, image_height)
X_train_transformed = sti.transform(X_train)

flat = FlattenCoordinates(65)
X_train_transformed = flat.transform(X_train)
print(X_train_transformed.shape)

(498, 2, 65, 2)
(996, 130)


In [18]:
model = RegressorChain(GradientBoostingRegressor(n_estimators=60, random_state=random_state))
print(X_train_transformed.shape, y_train.shape)
model.fit(X_train_transformed, y_train)

y_pred_train = model.predict(X_train_transformed)
train_mse = mean_squared_error(y_train, y_pred_train)

y_pred_val = model.predict(X_val_transformed)
test_mse = mean_squared_error(y_val, y_pred_val)

print(train_mse, test_mse)

(996, 130) (498, 2)


ValueError: Found input variables with inconsistent numbers of samples: [996, 498]